The video only shows little part of his code. Unfortunately I don't find his original code. I try to connect Dr.Yau yet received no reply. 

So I just replicate his code in slides here and try to make up the entire program by my knowledge in the following week. Yet it is very unlikely to finish because we don't know the data and the preprocessing way he used.

# VAR Part

## Index of Consumer Sentiment

In [ ]:
def tsplot2(y, title, lags=None, figsize=(12,8)):
    '''Examine the patterns of ACF and PACF, along with the time series plot and histogram.
    '''
    fig = plt.figure(figsize=figsize)
    layout = (2,2)
    ts_ax = plt.subplot2grid(layout,(0,0))
    hist_ax = plt.subplot2grid(layout,(0,1))
    acf_ax = plt.subplot2grid(layout,(1,0))
    pacf_ax = plt.subplot2grid(layout,(1,1))
    
    y.plot(ax=ts_ax)
    ts_ax.set_title(title, fontsize=14, fontweight='bold')
    y.plot(ax=hist_ax, kind='hist', bins=25)
    hist_ax.set_title('Histogram')
    smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
    smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
    [ax.set_xlim(0) for ax in [acf_ax, pacf_ax]]
    sns.despine()
    plt.tight_layout()
    return ts_ax, acf_ax, pacf_ax
    

## Transforming the Series

In [ ]:
series_transformed['UMCSENT'] = np.log(series.iloc[:,0])
series_transformed['beer'] = np.log(series.iloc[:,1])

## VAR Model Estimation and Output

In [ ]:
model = sm.tsa.VARMAX(y_train, order=(3,0), trend='c' )
model_result = model.fit(maxiter=1000, disp=False)
print(model_result.summmary())

## VAR Model Diagnostic

In [ ]:
model = sm.tsa.VARMAX(y_train, order=(3,0), trend='c')
model_result = model.fit(maxiter=1000, disp=False)
model_result.plot_diagnostics()

## VAR Model Selection

In [ ]:
aic = []
for i in range(5):
    i += 1
    model = sm.tsa.VARMAX(y_train, order=(i,0), trend='c')
    model_result = model.fit(maxiter=1000, disp=False)
    print('Order =', i)
    print('AIC:', model_result.aic)
    print('BIC:', model_result.bic)
    print('HQIC:', model_result.hqic)
    

## VAR Model Forecast

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error

VAR_forecast_beer = np.exp(z['beer'])*series['beer'][-3:]
VAR_forecast_UMCSENT = np.exp(z['UMCSENT'])*series['UMCSENT'][-3:]

rmse_beer = sqrt(mean_squared_error(series['beer'][-3:], VAR_forecast_UMCSENT))
rmse_UMCSENT = sqrt(mean_squared_error(series['UMCSENT'][-3:], VAR_forecast_UMCSENT))

# NN Part

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# Design the network architecture
model = Sequential()
model.add(LSTM(60,
               dropout=0.1
               recurrent_dropout=0.2,
               return_sequences = True,
               input_shape=(n_lookback,X_scaled_train_reshape.shape[2])))

model.add(LSTM(36))
model.add(Dense(X_scaled_train_reshape.shape[2]))
model.compile(loss='mae', optimizer='RMSprop')

# Model Training
n_epochs = 500
batchSize = 40
model.fit(X_scaled_train_reshape, y_scaled_train, epochs=n_epochs,
         batch_size=batchSize, verbose=0, shuffle=False)

# make a prediction
yhat_scale = model.predict(X_scaled_test_reshape)

# Inverse-scaling for forecast
inv_yhat = np.concatenate((X_scaled_test, yhat_scale), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)

# Model Evaluation: calculate RMSE
from math import sqrt
from sklearn.metrics import mean_square_error
print('Test RMSE: %.3f' % sqrt(mean_squared_error(y_test[:,0], inv_yhat[:,0])))
print('Test RMSE: %.3f' % sqrt(mean_squared_error(y_test[:,1], inv_yhat[:,1])))

## LSTM: Forecast Results

In [ ]:
# Model Evaluation: calculate RMSE
from math import sqrt
from sklearn.metrics import mean_square_error
print('Test RMSE: %.3f' % sqrt(mean_squared_error(y_test[:,0], inv_yhat[:,0])))
print('Test RMSE: %.3f' % sqrt(mean_squared_error(y_test[:,1], inv_yhat[:,1])))

In [ ]:
print(str(round((sqrt(mean_squared_error(y_test[:,0], inv_yhat[:,0]))/y_test[:,0].mean())*100,2)) + '%')
print(str(round((sqrt(mean_squared_error(y_test[:,0], inv_yhat[:,1]))/y_test[:,1].mean())*100,2)) + '%')